In [1]:
import numpy as np
import pandas as pd
from groq import Groq
import os
import time

In [2]:
df = pd.read_excel('../../data/raw/OMIEC_NEUROMORPHIC_07_08_24.xls')

In [3]:
GROQ_API_KEY='GROQ_API_KEY'

In [4]:
# groq
client = Groq(
    api_key=GROQ_API_KEY,
)

# respostas
responses_list = []

# todos os abstracts
total = len(df['Abstract'])
total_time = 0


for index, row in df.iterrows():
    start_time = time.time()

    # tempo
    print(f" {index} of {total}, remaining estimated time: {(total-index)*(total_time/(index+1))}")

    # chat request
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system",
                "content": "RolePlay as a bot seeking for polymers that are used in neuromorphic devices"
            },
            {
                "role": "user",
                #"content": "For the text I will input next, output only polymers used for neuromorphic devices. Do not output other types of polymers, and other kinds of materials, only the polymers. The output must be only the polymer material names separated by semicolons. Do not repeat the polymer name more than once. If no polymer was found, output 'None'"
                "content": "For the text I will input next, provide only the names of OMIEC polymers, also known as Organic Mixed Ionic Electronic Conductors, that are present in the text and are used for neuromorphic devices. Do not output other types of polymers, and other kinds of materials, only the OMIECs. If any polymer that meets the condition is found in the text, the output should be only the polymer material names separated by semicolons. If no polymer that meets the condition is found in the text, the output should be 'None'. Do not repeat the polymer name more than once. The output should not include the type of material. I want to use the output for future data analysis, so if the polymer name is written in full, replace it with its abbreviation."
            },
            {
                "role": "assistant",
                "content": "Hello, I will tell the polymers that are used in neuromorphic devices of any text you input next"
            },
            {
                "role": "user",
                "content": f"{row['Abstract']}"  # Use f-string to format the abstract
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    # coletar respostas
    response = ""

    # Stream the completion result and print the content
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""
        print(chunk.choices[0].delta.content or "", end="")

    # listas de respostas
    responses_list.append({'index': index, 'response': response})

    # tempo total
    total_time += time.time() - start_time

 0 of 54, remaining estimated time: 0.0
PEDOT;poly(3,4-ethylenedioxythiophene) 1 of 54, remaining estimated time: 24.928911566734314
P(NDI2OD-T2F); PFO-NDC; PMEG; PNP2TP; poly(3,4-ethylenedioxythiophene) (PEDOT) 2 of 54, remaining estimated time: 24.20369593302409
PEDOT:PSS 3 of 54, remaining estimated time: 22.441817700862885
PEDOT:PSS 4 of 54, remaining estimated time: 21.449074745178223
P3HT 5 of 54, remaining estimated time: 20.518308401107788
DPEPH; MPyP; P(NTf2)3 6 of 54, remaining estimated time: 23.508479527064733
N,N'-Bis(3-methylphenyl)-3,4,9,10-perylene tetracarboxylic diimide (PTC); Active Layer Blend of Poly(ethylenedioxythiophene) (PEDOT) and Poly(styrenesulfonate) (PSS) (PEDOT:PSS) 7 of 54, remaining estimated time: 23.393743604421616
Please input your text. 8 of 54, remaining estimated time: 22.351574473910862
P3HT; PEMA; Nafion 9 of 54, remaining estimated time: 21.198021411895752
Mel; PEDOT:PSS 10 of 54, remaining estimated time: 20.358959197998047
PEPDAT; PBTTTF 11 o

In [6]:
# DataFrame
df_with_responses = pd.DataFrame(responses_list)

# salvar para .csv
df_with_responses.to_excel('../../dataframes/OMIEC_NEUROMORPHIC_RESPONSES.xlsx', index=False)

#### Playground

In [ ]:
import requests
import time

df_with_responses = pd.DataFrame()

total = len(full_df['Abstract'])
total_time = 0

for index, row in full_df.iterrows():
    start_time = time.time()
    print(index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))
    payload = { 
        'prompt': """{
        'systemPrompt': 'RolePlay as a bot seeking for catalysts', 
        'user': "For the text I will input next, output only the catalyst materials used for the oxygen reduction reaction. Do not output precursors, electrolytes, and other kinds of materials, only the catalyst. The output must be only the catalyst material names separated by semicolons. Do not repeat the material name more than once. If no catalyst was found, output 'None'",
        'Assistant': 'Hello, I will tell the catalysts of any text you input next',
        'user_text':""" + row['Abstract'] + "}",
        "temperature":0.75,
        "topP":0.9,
        "maxTokens": 1000
    }
    #print(payload)
    response = requests.post('https://fumes-api.onrender.com/llama3', json=payload, stream=True)
    filtered_response = response.text.replace(' YOU CAN BUY ME COFFE! https://buymeacoffee.com/mygx', '')
    print(filtered_response)
    row['response'] = filtered_response
    df_with_responses = df_with_responses.append(row)
    time.sleep(2.5)
    total_time = total_time + time.time() - start_time
    print('Time for this loop:', time.time() - start_time, 'average loop time:', (total_time/(index+1)))
    #for chunk in response.iter_content(chunk_size=1024):  
    #    if chunk:
    #        print(chunk.decode('utf-8'))